# This script uses an API to retrieve storm report data from NOAA to assess the disaster caused by flash flooding in central Appalachia during July, 2022.

In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import re
import time
import gzip
import io
from datetime import datetime

import requests
import pandas as pd
from bs4 import BeautifulSoup

BASE_URL = "https://www.ncei.noaa.gov/pub/data/swdi/stormevents/csvfiles/"
OUT_DIR = "./storm_events_download_study"

UA = {
    "User-Agent": "stormevents-research/1.0 (contact: your_email@example.com)"
}

C_RE = re.compile(r"_c(\d{8})\.csv\.gz$", re.IGNORECASE)
GZ_RE = re.compile(r"\.csv\.gz$", re.IGNORECASE)

YEAR = 2022
DATE_START = "2022-07-26"
DATE_END   = "2022-07-28"

#  flood-related hazards.
KEEP_EVENT_TYPES = {"Flash Flood", "Flood", "Heavy Rain"}  # or None

DETAILS_TIME_COL = "BEGIN_DATE_TIME"

MERGE_KEY = "EVENT_ID"


def ensure_dir(p):
    os.makedirs(p, exist_ok=True)
    return p


def fetch_with_retries(url, headers, timeout=120, max_tries=5, backoff=2.0):
    last_err = None
    for i in range(1, max_tries + 1):
        try:
            r = requests.get(url, headers=headers, timeout=timeout)
            r.raise_for_status()
            return r.content
        except Exception as e:
            last_err = e
            sleep_s = backoff ** (i - 1)
            print("  download failed (try {0}/{1}): {2} | sleeping {3:.1f}s".format(
                i, max_tries, e, sleep_s
            ))
            time.sleep(sleep_s)
    raise RuntimeError("Failed to download after {0} tries: {1}".format(max_tries, url)) from last_err


def list_remote_files():
    idx = requests.get(BASE_URL, headers=UA, timeout=60)
    idx.raise_for_status()
    soup = BeautifulSoup(idx.text, "html.parser")
    links = [a.get("href") for a in soup.find_all("a") if a.get("href")]
    return [l for l in links if l and GZ_RE.search(l)]


def pick_latest_by_cdate(files):
    candidates = []
    no_c = []
    for f in files:
        m = C_RE.search(f)
        if m:
            candidates.append((datetime.strptime(m.group(1), "%Y%m%d"), f))
        else:
            no_c.append(f)

    if candidates:
        return max(candidates, key=lambda x: x[0])[1]
    if no_c:
        return sorted(no_c)[-1]
    return None


def read_gz_csv_bytes(content_bytes):
    with gzip.GzipFile(fileobj=io.BytesIO(content_bytes)) as gz:
        return pd.read_csv(gz, low_memory=False)


def download_one_year(year, links, kind="details"):
    if kind == "details":
        prefix = "StormEvents_details-ftp_v1.0_d{0}".format(year)
    elif kind == "locations":
        prefix = "StormEvents_locations-ftp_v1.0_d{0}".format(year)
    else:
        raise ValueError("kind must be details or locations")

    matches = [l for l in links if l.startswith(prefix) and l.endswith(".csv.gz")]
    chosen = pick_latest_by_cdate(matches)
    if not chosen:
        return None, None, None

    url = BASE_URL + chosen
    print("{0} {1}: downloading {2}".format(year, kind, chosen))
    content = fetch_with_retries(url, UA, timeout=180, max_tries=6, backoff=2.0)

    df = read_gz_csv_bytes(content)
    return chosen, df, content


def parse_noaa_dt(series):
    s = series.astype(str).str.strip()
    s = s.replace({"": pd.NA, "nan": pd.NA, "NaT": pd.NA})

    dt = pd.to_datetime(s, errors="coerce", format="%Y-%m-%d %H:%M:%S")
    if dt.notna().any():
        return dt
    dt2 = pd.to_datetime(s, errors="coerce", format="%d-%b-%y %H:%M:%S")
    if dt2.notna().any():
        return dt2
    dt3 = pd.to_datetime(s, errors="coerce", format="%d-%b-%y %I:%M:%S %p")
    if dt3.notna().any():
        return dt3
    return pd.to_datetime(s, errors="coerce")


def filter_details_to_period(df_details):
    if DETAILS_TIME_COL not in df_details.columns:
        raise ValueError("Time column not found in details: {0}".format(DETAILS_TIME_COL))

    d = df_details.copy()
    d["_dt"] = parse_noaa_dt(d[DETAILS_TIME_COL])

    start_dt = pd.to_datetime(DATE_START + " 00:00:00")
    end_dt = pd.to_datetime(DATE_END + " 23:59:59")

    d = d[d["_dt"].between(start_dt, end_dt, inclusive="both")].copy()

    if KEEP_EVENT_TYPES is not None and "EVENT_TYPE" in d.columns:
        d = d[d["EVENT_TYPE"].astype(str).str.strip().isin(KEEP_EVENT_TYPES)].copy()

    d.drop(columns=["_dt"], inplace=True, errors="ignore")
    return d


def filter_locations_by_event_ids(df_locations, event_ids):
    if MERGE_KEY not in df_locations.columns:
        raise ValueError("Merge key not found in locations: {0}".format(MERGE_KEY))
    return df_locations[df_locations[MERGE_KEY].isin(event_ids)].copy()


def main():
    raw_dir = ensure_dir(os.path.join(OUT_DIR, "raw"))
    raw_det_dir = ensure_dir(os.path.join(raw_dir, "details"))
    raw_loc_dir = ensure_dir(os.path.join(raw_dir, "locations"))
    filt_dir = ensure_dir(os.path.join(OUT_DIR, "filtered"))

    print("Fetching file list from NCEI...")
    links = list_remote_files()
    print("Found {0} .csv.gz entries in directory listing.".format(len(links)))

    chosen_det, df_det, raw_det = download_one_year(YEAR, links, kind="details")
    if df_det is None:
        raise RuntimeError("Details file not found for year {0}".format(YEAR))

    det_gz_path = os.path.join(raw_det_dir, chosen_det)
    with open(det_gz_path, "wb") as f:
        f.write(raw_det)
    print("Saved raw details gz:", det_gz_path)

    chosen_loc, df_loc, raw_loc = download_one_year(YEAR, links, kind="locations")
    if df_loc is None:
        raise RuntimeError("Locations file not found for year {0}".format(YEAR))

    loc_gz_path = os.path.join(raw_loc_dir, chosen_loc)
    with open(loc_gz_path, "wb") as f:
        f.write(raw_loc)
    print("Saved raw locations gz:", loc_gz_path)

    df_det_f = filter_details_to_period(df_det)

    if MERGE_KEY not in df_det_f.columns:
        raise ValueError("Merge key not found in filtered details: {0}".format(MERGE_KEY))

    event_ids = df_det_f[MERGE_KEY].dropna().unique().tolist()
    df_loc_f = filter_locations_by_event_ids(df_loc, event_ids)

    tag = "noaa_stormevents_{0}_{1}_{2}".format(
        YEAR,
        DATE_START.replace("-", ""),
        DATE_END.replace("-", "")
    )

    out_det = os.path.join(filt_dir, tag + "_details.csv")
    out_loc = os.path.join(filt_dir, tag + "_locations.csv")
    out_mrg = os.path.join(filt_dir, tag + "_details_locations_merged.csv")

    df_det_f.to_csv(out_det, index=False)
    df_loc_f.to_csv(out_loc, index=False)

    # Merging all data
    df_merged = df_loc_f.merge(df_det_f, on=MERGE_KEY, how="left")
    df_merged.to_csv(out_mrg, index=False)

    print("\n[OK] Filtered details rows:", len(df_det_f))
    print("[OK] Filtered locations rows:", len(df_loc_f))
    print("[OK] Wrote:", out_det)
    print("[OK] Wrote:", out_loc)
    print("[OK] Wrote:", out_mrg)
    print("\nDone.")


if __name__ == "__main__":
    main()


Fetching file list from NCEI...
Found 226 .csv.gz entries in directory listing.
2022 details: downloading StormEvents_details-ftp_v1.0_d2022_c20250721.csv.gz
Saved raw details gz: ./storm_events_download_study/raw/details/StormEvents_details-ftp_v1.0_d2022_c20250721.csv.gz
2022 locations: downloading StormEvents_locations-ftp_v1.0_d2022_c20250721.csv.gz
Saved raw locations gz: ./storm_events_download_study/raw/locations/StormEvents_locations-ftp_v1.0_d2022_c20250721.csv.gz

[OK] Filtered details rows: 177
[OK] Filtered locations rows: 277
[OK] Wrote: ./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv
[OK] Wrote: ./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_locations.csv
[OK] Wrote: ./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details_locations_merged.csv

Done.


In [4]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

EVENTS_CSV = "./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv"
STATES_SHP = "./ne_50m_admin_1_states_provinces_lakes/ne_50m_admin_1_states_provinces_lakes.shp"

COUNTIES_SHP = None

FOCUS_STATES = {"KENTUCKY", "WEST VIRGINIA", "VIRGINIA", "TENNESSEE", "OHIO"}

BBOX = dict(lon_min=-91.0, lon_max=-79.0, lat_min=34.0, lat_max=41.5)

OUT_DIR = "./outputs_appalachia_202207"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_MAP = os.path.join(OUT_DIR, "map_studyperiod_points_by_type.png")
OUT_HOURLY = os.path.join(OUT_DIR, "timeseries_hourly_counts.png")
OUT_HEATMAP = os.path.join(OUT_DIR, "heatmap_hour_by_day.png")
OUT_SUMMARY = os.path.join(OUT_DIR, "studyperiod_summary_table.csv")
OUT_COUNTY = os.path.join(OUT_DIR, "county_choropleth_counts.png")


def parse_noaa_dt(series):
    s = series.astype(str).str.strip()
    s = s.replace({"": pd.NA, "nan": pd.NA, "NaT": pd.NA})

    dt = pd.to_datetime(s, errors="coerce", format="%Y-%m-%d %H:%M:%S")
    if dt.notna().any():
        return dt
    dt2 = pd.to_datetime(s, errors="coerce", format="%d-%b-%y %H:%M:%S")
    if dt2.notna().any():
        return dt2
    dt3 = pd.to_datetime(s, errors="coerce", format="%d-%b-%y %I:%M:%S %p")
    if dt3.notna().any():
        return dt3

    return pd.to_datetime(s, errors="coerce")


def find_first_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        k = cand.lower()
        if k in cols:
            return cols[k]
    return None


def main():
    df = pd.read_csv(EVENTS_CSV, low_memory=False)
    print("[INFO] Loaded:", EVENTS_CSV, "rows=", len(df))

    lat_col = find_first_col(df, ["LATITUDE", "BEGIN_LAT", "LAT"])
    lon_col = find_first_col(df, ["LONGITUDE", "BEGIN_LON", "LON"])
    time_col = find_first_col(df, ["BEGIN_DATE_TIME", "BEGIN_DATETIME", "START_TIME"])
    etype_col = find_first_col(df, ["EVENT_TYPE"])
    state_col = find_first_col(df, ["STATE"])
    county_col = find_first_col(df, ["CZ_NAME", "COUNTY"])

    if lat_col is None or lon_col is None:
        raise ValueError("Could not find lat/lon columns in merged file.")
    if time_col is None:
        raise ValueError("Could not find BEGIN_DATE_TIME (or equivalent) in merged file.")
    if etype_col is None:
        print("[WARN] EVENT_TYPE not found; map legend may be limited.")
    if state_col is None:
        print("[WARN] STATE not found; state filtering will be skipped.")

    df = df.copy()
    df["dt"] = parse_noaa_dt(df[time_col])

    df["event_type"] = df[etype_col].astype(str).str.strip() if etype_col else "Unknown"

    # FIX: normalize STATE to uppercase to match FOCUS_STATES
    if state_col:
        df["state"] = df[state_col].astype(str).str.strip().str.upper()
    else:
        df["state"] = ""

    if county_col:
        df["county"] = df[county_col].astype(str).str.strip()
    else:
        df["county"] = ""

    print("[INFO] dt parsed non-null:", int(df["dt"].notna().sum()))
    print("[INFO] event types (top):")
    print(df["event_type"].value_counts().head(10))

    dff = df.copy()

    if state_col:
        dff = dff[dff["state"].isin(FOCUS_STATES)].copy()
        print("[DEBUG] States after state filter (top):")
        print(dff["state"].value_counts().head(10))
    else:
        print("[WARN] Skipping state filter (STATE column missing).")

    dff = dff[
        (dff[lon_col] >= BBOX["lon_min"]) & (dff[lon_col] <= BBOX["lon_max"]) &
        (dff[lat_col] >= BBOX["lat_min"]) & (dff[lat_col] <= BBOX["lat_max"])
    ].copy()

    print("[INFO] after state+bbox filter:", len(dff))
    if len(dff) > 0:
        print("[DEBUG] Lat range:", float(dff[lat_col].min()), float(dff[lat_col].max()))
        print("[DEBUG] Lon range:", float(dff[lon_col].min()), float(dff[lon_col].max()))

    # GeoDataFrame
    gdf = gpd.GeoDataFrame(
        dff,
        geometry=gpd.points_from_xy(dff[lon_col], dff[lat_col]),
        crs="EPSG:4326"
    )

    states = gpd.read_file(STATES_SHP).to_crs("EPSG:4326")

    name_col = find_first_col(states, ["name", "name_en", "name_long", "admin"]) or states.columns[0]

    states["_name_u"] = states[name_col].astype(str).str.strip().str.upper()

    states_focus = states[states["_name_u"].isin(FOCUS_STATES)].copy()
    if len(states_focus) == 0:
        print("[WARN] Could not match focus states in shapefile by name column:", name_col)
        states_focus = states.copy()

    summary = {}
    summary["n_points"] = int(len(gdf))
    summary["start_time"] = str(gdf["dt"].min()) if gdf["dt"].notna().any() else ""
    summary["end_time"] = str(gdf["dt"].max()) if gdf["dt"].notna().any() else ""

    vc = gdf["event_type"].value_counts()
    for k in vc.index[:10]:
        summary["count_" + k.replace(" ", "_")] = int(vc.loc[k])

    pd.DataFrame([summary]).to_csv(OUT_SUMMARY, index=False)
    print("[OK] wrote:", OUT_SUMMARY)

    fig, ax = plt.subplots(figsize=(10, 7))
    states_focus.boundary.plot(ax=ax, linewidth=1)

    if len(gdf) == 0:
        ax.text(
            0.5, 0.5,
            "No points after filters\nCheck state names + BBOX",
            transform=ax.transAxes, ha="center", va="center"
        )
    else:
        for etype, sub in gdf.groupby("event_type"):
            sub.plot(ax=ax, markersize=18, alpha=0.75, label=etype)
        ax.legend(loc="upper right", frameon=True, fontsize=9)

    ax.set_xlim(BBOX["lon_min"], BBOX["lon_max"])
    ax.set_ylim(BBOX["lat_min"], BBOX["lat_max"])
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_title("NOAA Storm Event Reports (Study Period)\nCentral Appalachia - July 26-27, 2022")

    plt.tight_layout()
    plt.savefig(OUT_MAP, dpi=300)
    plt.close(fig)
    print("[OK] saved:", OUT_MAP)


    if len(gdf) > 0 and gdf["dt"].notna().any():
        tmp = gdf.copy()
        tmp["hour"] = tmp["dt"].dt.floor("H")
        hourly = tmp.groupby("hour").size().sort_index()

        fig, ax = plt.subplots(figsize=(10, 4))
        ax.plot(hourly.index, hourly.values, marker="o")
        ax.set_title("Hourly Count of NOAA Reports (Study Period)")
        ax.set_xlabel("Time")
        ax.set_ylabel("Count")
        ax.grid(True, linewidth=0.3)
        plt.tight_layout()
        plt.savefig(OUT_HOURLY, dpi=300)
        plt.close(fig)
        print("[OK] saved:", OUT_HOURLY)
    else:
        print("[WARN] Skipping hourly time series (no datetime or no points).")

    if len(gdf) > 0 and gdf["dt"].notna().any():
        tmp = gdf.copy()
        tmp["day"] = tmp["dt"].dt.date.astype(str)
        tmp["hour_of_day"] = tmp["dt"].dt.hour

        value_col = "EVENT_ID" if "EVENT_ID" in tmp.columns else None
        if value_col is None:
            tmp["_one"] = 1
            value_col = "_one"

        pivot = tmp.pivot_table(
            index="hour_of_day",
            columns="day",
            values=value_col,
            aggfunc="count",
            fill_value=0
        )

        fig, ax = plt.subplots(figsize=(9, 5))
        im = ax.imshow(pivot.values, aspect="auto", origin="lower")
        ax.set_title("Report Density Heatmap (Hour x Day)")
        ax.set_xlabel("Day")
        ax.set_ylabel("Hour of day")

        ax.set_xticks(range(len(pivot.columns)))
        ax.set_xticklabels(pivot.columns, rotation=45, ha="right")
        ax.set_yticks(range(0, 24, 3))
        ax.set_yticklabels([str(h) for h in range(0, 24, 3)])

        fig.colorbar(im, ax=ax, label="Count")
        plt.tight_layout()
        plt.savefig(OUT_HEATMAP, dpi=300)
        plt.close(fig)
        print("[OK] saved:", OUT_HEATMAP)
    else:
        print("[WARN] Skipping heatmap (no datetime or no points).")

    if COUNTIES_SHP is not None and os.path.exists(COUNTIES_SHP) and len(gdf) > 0 and county_col is not None:
        counties = gpd.read_file(COUNTIES_SHP).to_crs("EPSG:4326")
        c_name = find_first_col(counties, ["name", "name_en", "name_long"]) or counties.columns[0]

        counts = gdf.groupby("county").size().reset_index(name="n_reports")
        counts["key"] = counts["county"].astype(str).str.lower()
        counties["key"] = counties[c_name].astype(str).str.lower()

        merged = counties.merge(counts, on="key", how="left")
        merged["n_reports"] = merged["n_reports"].fillna(0)

        fig, ax = plt.subplots(figsize=(10, 7))
        merged.plot(column="n_reports", ax=ax, legend=True)
        states_focus.boundary.plot(ax=ax, linewidth=0.8)

        ax.set_xlim(BBOX["lon_min"], BBOX["lon_max"])
        ax.set_ylim(BBOX["lat_min"], BBOX["lat_max"])
        ax.set_title("County-level Report Counts (Best-effort)")
        ax.set_axis_off()

        plt.tight_layout()
        plt.savefig(OUT_COUNTY, dpi=300)
        plt.close(fig)
        print("[OK] saved:", OUT_COUNTY)
    else:
        print("[INFO] county choropleth skipped (set COUNTIES_SHP to enable).")

    print("[DONE] All plots written to:", OUT_DIR)


if __name__ == "__main__":
    main()


[INFO] Loaded: ./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv rows= 177
[INFO] dt parsed non-null: 177
[INFO] event types (top):
event_type
Flash Flood    145
Flood           17
Heavy Rain      15
Name: count, dtype: int64
[DEBUG] States after state filter (top):
state
KENTUCKY         38
WEST VIRGINIA    21
VIRGINIA         12
Name: count, dtype: int64
[INFO] after state+bbox filter: 71
[DEBUG] Lat range: 36.7751 38.1121
[DEBUG] Lon range: -84.4713 -80.24
[OK] wrote: ./outputs_appalachia_202207/studyperiod_summary_table.csv
[OK] saved: ./outputs_appalachia_202207/map_studyperiod_points_by_type.png


/glade/derecho/scratch/npinkrah/tmp/ipykernel_133640/2634277672.py:204: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  tmp["hour"] = tmp["dt"].dt.floor("H")


[OK] saved: ./outputs_appalachia_202207/timeseries_hourly_counts.png
[OK] saved: ./outputs_appalachia_202207/heatmap_hour_by_day.png
[INFO] county choropleth skipped (set COUNTIES_SHP to enable).
[DONE] All plots written to: ./outputs_appalachia_202207


In [8]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import re
import pandas as pd

CSV_PATH = "./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv"
OUT_DIR = "./outputs_appalachia_202207"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_COLS_TXT = os.path.join(OUT_DIR, "noaa_columns_list.txt")
OUT_DETECTED = os.path.join(OUT_DIR, "noaa_detected_impact_fields.csv")


def norm(s: str) -> str:
    return re.sub(r"[^a-z0-9]+", "_", str(s).strip().lower()).strip("_")

def find_candidates(columns, patterns):
    """Return list of columns whose normalized name matches ANY regex pattern."""
    out = []
    for c in columns:
        nc = norm(c)
        for pat in patterns:
            if re.search(pat, nc):
                out.append(c)
                break
    return out

def preview_col(df, col, n=5):
    s = df[col]
    nonnull = s.dropna()
    if len(nonnull) == 0:
        return {"col": col, "dtype": str(s.dtype), "nonnull": 0, "examples": ""}
    ex = nonnull.astype(str).head(n).tolist()
    return {"col": col, "dtype": str(s.dtype), "nonnull": int(nonnull.shape[0]), "examples": " | ".join(ex)}

def is_damage_like(series: pd.Series) -> bool:
    """
    Detect NOAA-style damage strings: 0, 10K, 2.5M, 1B, etc.
    Works even if numeric.
    """
    s = series.dropna()
    if len(s) == 0:
        return False
    samp = s.astype(str).head(200).str.upper().str.strip()
    # digits with optional decimal and suffix K/M/B
    pat = re.compile(r"^\$?\s*\d+(\.\d+)?\s*[KMB]?$")
    hits = sum(bool(pat.match(x)) for x in samp)
    return hits >= max(3, int(0.05 * len(samp)))  # at least a few hits


def main():
    if not os.path.exists(CSV_PATH):
        raise FileNotFoundError(f"CSV not found: {CSV_PATH}")

    df = pd.read_csv(CSV_PATH, low_memory=False)
    cols = list(df.columns)

    print("\n=========================")
    print("FILE:", CSV_PATH)
    print("ROWS:", len(df))
    print("COLS:", len(cols))
    print("=========================\n")

    with open(OUT_COLS_TXT, "w", encoding="utf-8") as f:
        for c in cols:
            f.write(c + "\n")
    print("[OK] wrote column list:", OUT_COLS_TXT)

    print("---- ALL COLUMNS ----")
    for i, c in enumerate(cols, 1):
        print(f"{i:3d}. {c}")
    print("---------------------\n")

    patterns = {
        "injuries": [
            r"injur", r"injuries", r"injuries_direct", r"injuries_indirect",
            r"direct_inj", r"indirect_inj"
        ],
        "deaths": [
            r"death", r"deaths", r"deaths_direct", r"deaths_indirect",
            r"direct_death", r"indirect_death", r"fatal"
        ],
        "property_damage": [
            r"damage_property", r"property_damage", r"dmg_property", r"prop_dmg", r"damage_prop"
        ],
        "crop_damage": [
            r"damage_crops", r"crop_damage", r"dmg_crops", r"crop_dmg"
        ],
        "narrative_text": [
            r"narrative", r"episode_narrative", r"event_narrative", r"comments", r"remark"
        ],
        "location_text": [
            r"begin_location", r"end_location", r"location", r"cz_name", r"county", r"city", r"locality"
        ],
        "magnitude_severity": [
            r"magnitude", r"mag", r"f_scale", r"tor_f_scale", r"fujita", r"cz_type",
            r"flood_cause", r"severity", r"source"
        ],
    }

    detected = []
    for group, pats in patterns.items():
        cands = find_candidates(cols, pats)
        if cands:
            for c in cands:
                detected.append((group, c))

    damage_like_cols = []
    for c in cols:
        try:
            if is_damage_like(df[c]):
                damage_like_cols.append(c)
        except Exception:
            continue

    rows = []
    print("==== DETECTED IMPACT-RELATED FIELDS (pattern-based) ====")
    if not detected:
        print("None detected by name patterns.")
    else:
        for group, c in detected:
            info = preview_col(df, c, n=6)
            rows.append({"group": group, **info})
            print(f"[{group}] {c} | dtype={info['dtype']} | nonnull={info['nonnull']}")
            if info["examples"]:
                print("   examples:", info["examples"])
    print("=======================================================\n")

    print("==== DAMAGE-LIKE COLUMNS ====")
    if not damage_like_cols:
        print("None detected by damage-string scan.")
    else:
        for c in damage_like_cols:
            info = preview_col(df, c, n=6)
            rows.append({"group": "damage_like_scan", **info})
            print(f"[damage_like_scan] {c} | dtype={info['dtype']} | nonnull={info['nonnull']}")
            if info["examples"]:
                print("   examples:", info["examples"])
    print("==========================\n")

    if rows:
        out = pd.DataFrame(rows).drop_duplicates(subset=["group", "col"])
        out.to_csv(OUT_DETECTED, index=False)
        print("[OK] wrote detected-field summary:", OUT_DETECTED)
    else:
        print("[WARN] No detected fields to write.")

    def best_guess(group_names):
        cands = [r["col"] for r in rows if r["group"] in group_names]
        # Prefer columns with most non-null values
        best = None
        best_n = -1
        for c in cands:
            nn = int(df[c].notna().sum())
            if nn > best_n:
                best_n = nn
                best = c
        return best

    guess = {
        "injuries": best_guess(["injuries"]),
        "deaths": best_guess(["deaths"]),
        "property_damage": best_guess(["property_damage", "damage_like_scan"]),
        "crop_damage": best_guess(["crop_damage", "damage_like_scan"]),
        "event_narrative": best_guess(["narrative_text"]),
        "location_text": best_guess(["location_text"]),
    }

    for k, v in guess.items():
        print(f"{k:16s}: {v}")
    print("==================================================\n")

 
if __name__ == "__main__":
    main()



FILE: ./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv
ROWS: 177
COLS: 51

[OK] wrote column list: ./outputs_appalachia_202207/noaa_columns_list.txt
---- ALL COLUMNS ----
  1. BEGIN_YEARMONTH
  2. BEGIN_DAY
  3. BEGIN_TIME
  4. END_YEARMONTH
  5. END_DAY
  6. END_TIME
  7. EPISODE_ID
  8. EVENT_ID
  9. STATE
 10. STATE_FIPS
 11. YEAR
 12. MONTH_NAME
 13. EVENT_TYPE
 14. CZ_TYPE
 15. CZ_FIPS
 16. CZ_NAME
 17. WFO
 18. BEGIN_DATE_TIME
 19. CZ_TIMEZONE
 20. END_DATE_TIME
 21. INJURIES_DIRECT
 22. INJURIES_INDIRECT
 23. DEATHS_DIRECT
 24. DEATHS_INDIRECT
 25. DAMAGE_PROPERTY
 26. DAMAGE_CROPS
 27. SOURCE
 28. MAGNITUDE
 29. MAGNITUDE_TYPE
 30. FLOOD_CAUSE
 31. CATEGORY
 32. TOR_F_SCALE
 33. TOR_LENGTH
 34. TOR_WIDTH
 35. TOR_OTHER_WFO
 36. TOR_OTHER_CZ_STATE
 37. TOR_OTHER_CZ_FIPS
 38. TOR_OTHER_CZ_NAME
 39. BEGIN_RANGE
 40. BEGIN_AZIMUTH
 41. BEGIN_LOCATION
 42. END_RANGE
 43. END_AZIMUTH
 44. END_LOCATION
 45. BEGIN_LAT
 46. BEGIN_LON
 47. END_L

In [14]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
EMDS 501 / Natural & Technological Disaster Risks
"Other side" of NOAA Storm Events for the study period:
IMPACTS + SEVERITY + HUMAN/Economic dimensions (no interpolation).

Inputs:
  - Your merged NOAA details+locations CSV (study period)

Outputs (in OUT_DIR):
  1) impact_summary_totals.txt
  2) bar_damage_by_state.png                 (total $ property damage by state)
  3) bar_damage_by_top_counties.png          (top counties by total $ property damage)
  4) map_points_damage_bubbles.png           (points map sized by property damage, colored by event type)
  5) map_state_property_damage.png           (state choropleth by total $ property damage + table below)
  6) bar_flood_cause_counts.png              (counts by FLOOD_CAUSE)
  7) bar_source_counts.png                   (counts by SOURCE)
  8) bar_top_keywords_narratives.png         (top words from narratives, simple frequency)
  9) pareto_damage_by_county.png             (damage concentration / Pareto)

Notes:
- Parses DAMAGE_PROPERTY / DAMAGE_CROPS strings like "0.20K", "500.00K", "1.2M", "0".
- If injuries/deaths are all zeros (common), it still prints totals (useful for your writeup).
- Spatial plots use a 2-row layout (map on top, table below) so tables never cover the map.
- If the Natural Earth states shapefile is missing, it auto-downloads + unzips it.
"""

import os
import re
import zipfile
import urllib.request

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

from matplotlib.gridspec import GridSpec


EVENTS_CSV = "./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv"

STATES_SHP = "./ne_50m_admin_1_states_provinces_lakes/ne_50m_admin_1_states_provinces_lakes.shp"

FOCUS_STATES = {"KENTUCKY", "WEST VIRGINIA", "VIRGINIA", "TENNESSEE", "OHIO"}

BBOX = dict(lon_min=-91.0, lon_max=-79.0, lat_min=34.0, lat_max=41.5)

# Flood events only 
KEEP_EVENT_TYPES = {"FLASH FLOOD", "FLOOD"}

OUT_DIR = "./outputs_appalachia_202207_impacts"
os.makedirs(OUT_DIR, exist_ok=True)

OUT_TXT_SUMMARY     = os.path.join(OUT_DIR, "impact_summary_totals.txt")
OUT_BAR_STATE_DMG   = os.path.join(OUT_DIR, "bar_damage_by_state.png")
OUT_BAR_COUNTY_DMG  = os.path.join(OUT_DIR, "bar_damage_by_top_counties.png")
OUT_MAP_BUBBLES_DMG = os.path.join(OUT_DIR, "map_points_damage_bubbles.png")
OUT_MAP_STATE_DMG   = os.path.join(OUT_DIR, "map_state_property_damage.png")
OUT_BAR_CAUSE       = os.path.join(OUT_DIR, "bar_flood_cause_counts.png")
OUT_BAR_SOURCE      = os.path.join(OUT_DIR, "bar_source_counts.png")
OUT_BAR_WORDS       = os.path.join(OUT_DIR, "bar_top_keywords_narratives.png")
OUT_PARETO_COUNTY   = os.path.join(OUT_DIR, "pareto_damage_by_county.png")

TOP_N_COUNTIES = 10
TOP_N_WORDS    = 15

NE_URL = "https://naturalearth.s3.amazonaws.com/50m_cultural/ne_50m_admin_1_states_provinces.zip"
NE_DIR = "./ne_50m_admin_1_states_provinces"
NE_ZIP = os.path.join(NE_DIR, "ne_50m_admin_1_states_provinces.zip")
NE_SHP = os.path.join(NE_DIR, "ne_50m_admin_1_states_provinces.shp")


def ensure_naturalearth_states():

    os.makedirs(NE_DIR, exist_ok=True)
    if os.path.exists(NE_SHP):
        return NE_SHP

    print("[INFO] Downloading:", NE_URL)
    urllib.request.urlretrieve(NE_URL, NE_ZIP)

    print("[INFO] Unzipping:", NE_ZIP)
    with zipfile.ZipFile(NE_ZIP, "r") as z:
        z.extractall(NE_DIR)

    if not os.path.exists(NE_SHP):
        raise FileNotFoundError("Natural Earth unzip succeeded but .shp not found: " + NE_SHP)

    return NE_SHP


def find_first_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        k = cand.lower()
        if k in cols:
            return cols[k]
    return None


def parse_noaa_dt(series):
    s = series.astype(str).str.strip()
    s = s.replace({"": pd.NA, "nan": pd.NA, "NaT": pd.NA})

    dt = pd.to_datetime(s, errors="coerce", format="%Y-%m-%d %H:%M:%S")
    if dt.notna().any():
        return dt
    dt2 = pd.to_datetime(s, errors="coerce", format="%d-%b-%y %H:%M:%S")
    if dt2.notna().any():
        return dt2
    dt3 = pd.to_datetime(s, errors="coerce", format="%d-%b-%y %I:%M:%S %p")
    if dt3.notna().any():
        return dt3

    return pd.to_datetime(s, errors="coerce")


def is_us_states_layer(gdf_states):
    if "admin" in gdf_states.columns:
        return gdf_states[gdf_states["admin"].astype(str) == "United States of America"].copy()
    return gdf_states.copy()


def normalize_state_name(s):
    return str(s).strip().upper()


def safe_title(s):
    return str(s).strip().title()


def damage_to_usd(x):
    """
    NOAA Storm Events damage format examples:
      "0", "0.20K", "500.00K", "1.2M", "2B"
    Returns float USD.
    """
    if pd.isna(x):
        return 0.0
    s = str(x).strip().upper().replace(",", "")
    if s in ("", "NAN", "NONE"):
        return 0.0

    s = s.lstrip("$").strip()

    # pure numeric?
    try:
        return float(s)
    except Exception:
        pass

    m = re.match(r"^(\d+(\.\d+)?)\s*([KMB])?$", s)
    if not m:
        s2 = re.sub(r"\s+", "", s)
        m2 = re.match(r"^(\d+(\.\d+)?)\s*([KMB])?$", s2)
        if not m2:
            return 0.0
        m = m2

    val = float(m.group(1))
    suf = m.group(3)
    mult = 1.0
    if suf == "K":
        mult = 1e3
    elif suf == "M":
        mult = 1e6
    elif suf == "B":
        mult = 1e9
    return val * mult


def fmt_money(x):
    x = float(x)
    if x >= 1e9:
        return f"${x/1e9:.2f}B"
    if x >= 1e6:
        return f"${x/1e6:.2f}M"
    if x >= 1e3:
        return f"${x/1e3:.2f}K"
    return f"${x:.0f}"


def make_table(ax, header, rows, fontsize=10):
    data = [header] + rows
    t = ax.table(cellText=data, cellLoc="center", loc="center")
    t.auto_set_font_size(False)
    t.set_fontsize(fontsize)
    for j in range(len(header)):
        t[(0, j)].set_text_props(fontweight="bold")
    ax.axis("off")
    return t


def tokenize(text):
    # simple tokenization for narrative keywords
    text = str(text).lower()
    text = re.sub(r"[^a-z\s]", " ", text)
    toks = [w for w in text.split() if len(w) >= 4]
    stop = {
        "with","that","this","from","were","over","into","also","fell","kept","near","during",
        "resulted","county","counties","water","rain","flood","flooding","flash","roads",
        "roadway","local","creeks","streams","banks","community","early","morning","hours",
        "evening","night","area","state","along","continued","moved","across","remained",
        "period","spots","inches","several","including","entered","first","floor","building"
    }
    return [w for w in toks if w not in stop]


def bar_with_labels(ax, labels, values, label_fmt=None, rotation=25):
    x = np.arange(len(labels))
    ax.bar(x, values)
    ax.set_xticks(x)
    ax.set_xticklabels(labels, rotation=rotation, ha="right")
    for i, v in enumerate(values):
        txt = label_fmt(v) if label_fmt else str(v)
        ax.text(i, v, txt, ha="center", va="bottom", fontsize=9)


def main():
    if not os.path.exists(EVENTS_CSV):
        raise FileNotFoundError("Missing EVENTS_CSV: " + EVENTS_CSV)

    states_shp_path = STATES_SHP
    if not os.path.exists(states_shp_path):
        states_shp_path = ensure_naturalearth_states()

    df = pd.read_csv(EVENTS_CSV, low_memory=False)
    print("[INFO] Loaded:", EVENTS_CSV, "rows=", len(df))

    lat_col = find_first_col(df, ["LATITUDE", "BEGIN_LAT", "LAT", "BEGIN_LATITUDE"])
    lon_col = find_first_col(df, ["LONGITUDE", "BEGIN_LON", "LON", "BEGIN_LONGITUDE"])
    time_col = find_first_col(df, ["BEGIN_DATE_TIME"])
    etype_col = find_first_col(df, ["EVENT_TYPE"])
    state_col = find_first_col(df, ["STATE"])
    county_col = find_first_col(df, ["CZ_NAME", "COUNTY"])

    dmgp_col = find_first_col(df, ["DAMAGE_PROPERTY"])
    dmgc_col = find_first_col(df, ["DAMAGE_CROPS"])
    injd_col = find_first_col(df, ["INJURIES_DIRECT"])
    inji_col = find_first_col(df, ["INJURIES_INDIRECT"])
    dead_col = find_first_col(df, ["DEATHS_DIRECT"])
    deai_col = find_first_col(df, ["DEATHS_INDIRECT"])
    cause_col = find_first_col(df, ["FLOOD_CAUSE"])
    src_col   = find_first_col(df, ["SOURCE"])
    epn_col   = find_first_col(df, ["EPISODE_NARRATIVE"])
    evn_col   = find_first_col(df, ["EVENT_NARRATIVE"])

    for must in [lat_col, lon_col, etype_col, state_col, dmgp_col]:
        if must is None:
            raise ValueError("Missing a required column. Check the CSV schema.")

    dff = df.copy()
    dff["dt"] = parse_noaa_dt(dff[time_col]) if time_col else pd.NaT
    dff["state_u"] = dff[state_col].map(normalize_state_name)
    dff["state_t"] = dff[state_col].map(safe_title)
    dff["etype_u"] = dff[etype_col].astype(str).str.strip().str.upper()
    dff["etype_t"] = dff[etype_col].astype(str).str.strip()
    dff["county"]  = dff[county_col].astype(str).str.strip().str.title() if county_col else ""

    dff = dff[dff["state_u"].isin(FOCUS_STATES)].copy()

    dff = dff[
        (pd.to_numeric(dff[lon_col], errors="coerce") >= BBOX["lon_min"]) &
        (pd.to_numeric(dff[lon_col], errors="coerce") <= BBOX["lon_max"]) &
        (pd.to_numeric(dff[lat_col], errors="coerce") >= BBOX["lat_min"]) &
        (pd.to_numeric(dff[lat_col], errors="coerce") <= BBOX["lat_max"])
    ].copy()

    if KEEP_EVENT_TYPES is not None:
        keep_u = {k.strip().upper() for k in KEEP_EVENT_TYPES}
        dff = dff[dff["etype_u"].isin(keep_u)].copy()

    print("[INFO] After filters rows:", len(dff))
    print("[INFO] Event types:", dff["etype_t"].value_counts().to_dict())

    # Damage numeric
    dff["prop_usd"] = dff[dmgp_col].map(damage_to_usd).astype(float)
    dff["crop_usd"] = dff[dmgc_col].map(damage_to_usd).astype(float) if dmgc_col else 0.0

    # Injury/death totals
    inj_direct = float(dff[injd_col].sum()) if injd_col else 0.0
    inj_indir  = float(dff[inji_col].sum()) if inji_col else 0.0
    dea_direct = float(dff[dead_col].sum()) if dead_col else 0.0
    dea_indir  = float(dff[deai_col].sum()) if deai_col else 0.0

    total_prop = float(dff["prop_usd"].sum())
    total_crop = float(dff["crop_usd"].sum())

    # Write summary
    with open(OUT_TXT_SUMMARY, "w", encoding="utf-8") as f:
        f.write("Study Period Impact Summary\n")
        f.write("===========================\n")
        f.write(f"Rows (reports): {len(dff)}\n")
        f.write(f"Total property damage: {fmt_money(total_prop)}\n")
        f.write(f"Total crop damage:     {fmt_money(total_crop)}\n")
        f.write(f"Injuries (direct):     {inj_direct:.0f}\n")
        f.write(f"Injuries (indirect):   {inj_indir:.0f}\n")
        f.write(f"Deaths (direct):       {dea_direct:.0f}\n")
        f.write(f"Deaths (indirect):     {dea_indir:.0f}\n")
    print("[OK] Saved:", OUT_TXT_SUMMARY)

    # Geo points
    gdf_pts = gpd.GeoDataFrame(
        dff,
        geometry=gpd.points_from_xy(dff[lon_col], dff[lat_col]),
        crs="EPSG:4326"
    )

    # Load states outlines (focus)
    gdf_states = is_us_states_layer(gpd.read_file(states_shp_path))

    name_col = find_first_col(gdf_states, ["name", "name_en", "name_long", "NAME", "NAME_EN"])
    if name_col is None:

        non_geom_cols = [c for c in gdf_states.columns if c != "geometry"]
        if not non_geom_cols:
            raise ValueError("States shapefile has no usable name columns.")
        name_col = non_geom_cols[0]

    gdf_states = gdf_states.copy()
    gdf_states["STATE_T"] = gdf_states[name_col].astype(str).str.strip().str.title()
    gdf_states["STATE_U"] = gdf_states[name_col].astype(str).str.strip().str.upper()

    states_focus = gdf_states[gdf_states["STATE_U"].isin(FOCUS_STATES)].copy()
    if len(states_focus) == 0:
        states_focus = gdf_states.copy()

    dmg_by_state = dff.groupby("state_t")["prop_usd"].sum().sort_values(ascending=False)
    fig, ax = plt.subplots(figsize=(10, 5))
    bar_with_labels(ax, list(dmg_by_state.index), list(dmg_by_state.values), label_fmt=fmt_money, rotation=25)
    ax.set_title("Total Property Damage by State \nCentral Appalachia - July 26-28, 2022", fontsize=14)
    ax.set_ylabel("Property damage (USD)")
    ax.grid(True, axis="y", linewidth=0.3)
    plt.tight_layout()
    plt.savefig(OUT_BAR_STATE_DMG, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("[OK] Saved:", OUT_BAR_STATE_DMG)

    if county_col:
        dmg_by_cty = dff.groupby("county")["prop_usd"].sum().sort_values(ascending=False).head(TOP_N_COUNTIES)
        fig, ax = plt.subplots(figsize=(11, 5))
        bar_with_labels(ax, list(dmg_by_cty.index), list(dmg_by_cty.values), label_fmt=fmt_money, rotation=30)
        ax.set_title(
            f"Top {TOP_N_COUNTIES} Counties by Property Damage\nCentral Appalachia - July 26-28, 2022",
            fontsize=14
        )
        ax.set_ylabel("Property damage (USD)")
        ax.grid(True, axis="y", linewidth=0.3)
        plt.tight_layout()
        plt.savefig(OUT_BAR_COUNTY_DMG, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print("[OK] Saved:", OUT_BAR_COUNTY_DMG)

    fig, ax = plt.subplots(figsize=(14, 9))
    states_focus.to_crs("EPSG:4326").boundary.plot(ax=ax, linewidth=1.4, edgecolor="0.25")

    vals = gdf_pts["prop_usd"].values.astype(float)
    if len(vals) > 0 and np.isfinite(vals).any():
        p95 = np.nanpercentile(vals, 95)
        p95 = max(float(p95), 1.0)
        size = 30 + 220 * np.clip(vals / p95, 0, 1)  # 30..250
    else:
        size = np.full(len(gdf_pts), 30.0)

    etype_colors = {"FLASH FLOOD": "tab:blue", "FLOOD": "tab:orange"}
    colors = gdf_pts["etype_u"].map(etype_colors).fillna("tab:gray")

    ax.scatter(
        gdf_pts.geometry.x,
        gdf_pts.geometry.y,
        s=size,
        c=colors,
        alpha=0.75,
        edgecolors="none"
    )

    ax.set_xlim(BBOX["lon_min"], BBOX["lon_max"])
    ax.set_ylim(BBOX["lat_min"], BBOX["lat_max"])
    ax.set_title(
        "NOAA Flood Reports Sized by Property Damage (Study Period)\nCentral Appalachia - July 26-28, 2022",
        fontsize=20
    )
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_xlabel("")
    ax.set_ylabel("")
    ax.set_frame_on(False)

    # Legend: event type
    for k, c in etype_colors.items():
        ax.scatter([], [], s=60, c=c, label=k.title(), alpha=0.9)

    # Legend: bubble sizes (relative damage)
    for frac, lab in [(0.25, "Low"), (0.60, "Medium"), (1.00, "High")]:
        ax.scatter([], [], s=30 + 220 * frac, c="0.3", label=f"Damage: {lab}", alpha=0.3)

    ax.legend(loc="upper right", frameon=True, title="Legend")

    plt.tight_layout()
    plt.savefig(OUT_MAP_BUBBLES_DMG, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("[OK] Saved:", OUT_MAP_BUBBLES_DMG)

    dmg_state = dff.groupby("state_u")["prop_usd"].sum().sort_values(ascending=False)
    df_merge = dmg_state.reset_index()
    df_merge.columns = ["STATE_U", "Prop_USD"]

    states_plot = states_focus.copy()
    states_plot = states_plot.merge(df_merge, on="STATE_U", how="left")
    states_plot["Prop_USD"] = states_plot["Prop_USD"].fillna(0.0)

    # log scale for better visualization (keep labels as real dollars)
    states_plot["Prop_USD_LOG10"] = np.log10(states_plot["Prop_USD"].clip(lower=1.0))

    fig = plt.figure(figsize=(14, 11))
    gs = GridSpec(nrows=2, ncols=1, height_ratios=[4.2, 1.4], hspace=0.05)

    ax_map = fig.add_subplot(gs[0, 0])
    ax_tbl = fig.add_subplot(gs[1, 0])

    states_plot.plot(
        column="Prop_USD_LOG10",
        cmap="YlOrRd",
        linewidth=1.0,
        edgecolor="0.6",
        legend=True,
        legend_kwds={"label": r"$\log_{10}(\mathrm{Total\ property\ damage\ [USD]})$"},
        ax=ax_map
    )

    for _, row in states_plot.iterrows():
        if row["geometry"].is_empty:
            continue
        pt = row["geometry"].representative_point()
        ax_map.text(
            pt.x,
            pt.y,
            fmt_money(row["Prop_USD"]),
            ha="center",
            fontsize=12,
            fontweight="bold"
        )

    ax_map.set_title(
        "Property Damage by State\nCentral Appalachia - July 26-28, 2022",
        fontsize=22
    )
    ax_map.set_xticks([])
    ax_map.set_yticks([])
    ax_map.set_xlabel("")
    ax_map.set_ylabel("")
    ax_map.set_frame_on(False)

    top3 = states_plot.sort_values("Prop_USD", ascending=False).head(3)
    rows = [[r["STATE_T"], fmt_money(r["Prop_USD"])] for _, r in top3.iterrows()]
    make_table(ax_tbl, ["State", "Total Property Damage"], rows, fontsize=12)

    plt.savefig(OUT_MAP_STATE_DMG, dpi=300, bbox_inches="tight")
    plt.close(fig)
    print("[OK] Saved:", OUT_MAP_STATE_DMG)

    if cause_col:
        cause = dff[cause_col].fillna("Unknown").astype(str).str.strip()
        cause_counts = cause.value_counts().head(12)

        fig, ax = plt.subplots(figsize=(11, 5))
        bar_with_labels(ax, list(cause_counts.index), list(cause_counts.values), label_fmt=lambda v: str(int(v)), rotation=25)
        ax.set_title("Flood Cause (Counts) - Study Period\nCentral Appalachia - July 26-28, 2022", fontsize=14)
        ax.set_ylabel("Number of reports")
        ax.grid(True, axis="y", linewidth=0.3)
        plt.tight_layout()
        plt.savefig(OUT_BAR_CAUSE, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print("[OK] Saved:", OUT_BAR_CAUSE)

    if src_col:
        src = dff[src_col].fillna("Unknown").astype(str).str.strip()
        src_counts = src.value_counts().head(12)

        fig, ax = plt.subplots(figsize=(11, 5))
        bar_with_labels(ax, list(src_counts.index), list(src_counts.values), label_fmt=lambda v: str(int(v)), rotation=25)
        ax.set_title("NOAA Storm Events Reporting Source (Counts)\nCentral Appalachia - July 26-28, 2022", fontsize=14)
        ax.set_ylabel("Number of reports")
        ax.grid(True, axis="y", linewidth=0.3)
        plt.tight_layout()
        plt.savefig(OUT_BAR_SOURCE, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print("[OK] Saved:", OUT_BAR_SOURCE)

    texts = []
    if epn_col:
        texts.append(dff[epn_col].fillna("").astype(str))
    if evn_col:
        texts.append(dff[evn_col].fillna("").astype(str))

    if texts:
        all_text = " ".join(pd.concat(texts, axis=0).tolist())
        toks = tokenize(all_text)
        if toks:
            vc = pd.Series(toks).value_counts().head(TOP_N_WORDS)

            fig, ax = plt.subplots(figsize=(11, 5))
            bar_with_labels(ax, list(vc.index), list(vc.values), label_fmt=lambda v: str(int(v)), rotation=25)
            ax.set_title("Top Narrative Keywords (Simple Frequency)\nCentral Appalachia - July 26-28, 2022", fontsize=14)
            ax.set_ylabel("Count")
            ax.grid(True, axis="y", linewidth=0.3)
            plt.tight_layout()
            plt.savefig(OUT_BAR_WORDS, dpi=300, bbox_inches="tight")
            plt.close(fig)
            print("[OK] Saved:", OUT_BAR_WORDS)
        else:
            print("[WARN] No tokens extracted from narratives (unexpected).")

    # county concentration of property damage
    if county_col:
        tmp = dff.groupby("county")["prop_usd"].sum().sort_values(ascending=False)
        pareto = tmp.reset_index()
        pareto.columns = ["county", "prop_usd"]
        denom = max(float(tmp.sum()), 1.0)
        pareto["cum_pct"] = 100.0 * pareto["prop_usd"].cumsum() / denom

        top = pareto.head(15)

        fig, ax1 = plt.subplots(figsize=(12, 5))
        x = np.arange(len(top["county"]))
        ax1.bar(x, top["prop_usd"])
        ax1.set_ylabel("Property damage (USD)")
        ax1.set_xticks(x)
        ax1.set_xticklabels(top["county"], rotation=30, ha="right")
        ax1.grid(True, axis="y", linewidth=0.3)

        ax2 = ax1.twinx()
        ax2.plot(x, top["cum_pct"], marker="o")
        ax2.set_ylabel("Cumulative % of total damage")
        ax2.set_ylim(0, 105)

        ax1.set_title("County Concentration of Property Damage \nCentral Appalachia - July 26-28, 2022")
        plt.tight_layout()
        plt.savefig(OUT_PARETO_COUNTY, dpi=300, bbox_inches="tight")
        plt.close(fig)
        print("[OK] Saved:", OUT_PARETO_COUNTY)

    print("[DONE] Impact outputs written to:", OUT_DIR)


if __name__ == "__main__":
    main()


[INFO] Loaded: ./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv rows= 177
[INFO] After filters rows: 64
[INFO] Event types: {'Flash Flood': 53, 'Flood': 11}
[OK] Saved: ./outputs_appalachia_202207_impacts/impact_summary_totals.txt
[OK] Saved: ./outputs_appalachia_202207_impacts/bar_damage_by_state.png
[OK] Saved: ./outputs_appalachia_202207_impacts/bar_damage_by_top_counties.png
[OK] Saved: ./outputs_appalachia_202207_impacts/map_points_damage_bubbles.png
[OK] Saved: ./outputs_appalachia_202207_impacts/map_state_property_damage.png
[OK] Saved: ./outputs_appalachia_202207_impacts/bar_flood_cause_counts.png
[OK] Saved: ./outputs_appalachia_202207_impacts/bar_source_counts.png
[OK] Saved: ./outputs_appalachia_202207_impacts/bar_top_keywords_narratives.png
[OK] Saved: ./outputs_appalachia_202207_impacts/pareto_damage_by_county.png
[DONE] Impact outputs written to: ./outputs_appalachia_202207_impacts


In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

EVENTS_CSV = "./storm_events_download_study/filtered/noaa_stormevents_2022_20220726_20220728_details.csv"

df = pd.read_csv(EVENTS_CSV, low_memory=False)

def find_first_col(df, candidates):
    cols = {c.lower(): c for c in df.columns}
    for cand in candidates:
        if cand.lower() in cols:
            return cols[cand.lower()]
    return None

dead_col = find_first_col(df, ["DEATHS_DIRECT"])
deai_col = find_first_col(df, ["DEATHS_INDIRECT"])
state_col = find_first_col(df, ["STATE"])

dea_direct = float(df[dead_col].sum()) if dead_col else 0
dea_indir  = float(df[deai_col].sum()) if deai_col else 0

print("Direct deaths:", int(dea_direct))
print("Indirect deaths:", int(dea_indir))
labels = ["Direct", "Indirect"]
vals = [dea_direct, dea_indir]

fig, ax = plt.subplots(figsize=(6,4))
x = np.arange(len(labels))

ax.bar(x, vals)
ax.set_xticks(x)
ax.set_xticklabels(labels)

for i, v in enumerate(vals):
    ax.text(i, v, str(int(v)), ha="center", va="bottom")

ax.set_title("Fatalities (NOAA Storm Events)\nJuly 26–28, 2022")
ax.set_ylabel("Number of deaths")
ax.grid(True, axis="y", linewidth=0.3)

plt.close()

deaths_by_state = df.groupby(state_col)[dead_col].sum().sort_values(ascending=False)

fig, ax = plt.subplots(figsize=(8,4))

x = np.arange(len(deaths_by_state))
ax.bar(x, deaths_by_state.values)

ax.set_xticks(x)
ax.set_xticklabels(deaths_by_state.index, rotation=30, ha="right")

for i, v in enumerate(deaths_by_state.values):
    ax.text(i, v, str(int(v)), ha="center", va="bottom")

ax.set_title("Direct Fatalities by State\nJuly 26–28, 2022")
ax.set_ylabel("Deaths")

ax.grid(True, axis="y", linewidth=0.3)
plt.tight_layout()
plt.close()


Direct deaths: 41
Indirect deaths: 1
